In [1]:
import pandas as pd
import numpy as np

## Getting the data

In [5]:
tweets = pd.read_csv('dataset/general-tweets.txt',sep="\t",header=None)
tweets.columns = ['label','tweets']
tweets.head()

,label,tweets
0,NOT,Bumping dj sefs mixtape nowww this is my music...
1,NOT,#ieroween THE STORY OF IEROWEEN! THE VIDEO ->>...
2,NOT,trick or treating at the mall today; ZOO! last...
3,NOT,@Ussk81 PMSL!!! I try not to stare but I can't...
4,NOT,@Sc0rpi0n676 btw - is there a remote chance i ...


In [7]:
x_train = tweets['tweets']
y_train = tweets['label']

In [8]:
tweets_test = pd.read_csv('dataset/keyword-tweets.txt',sep="\t",header=None)
tweets_test.columns = ['label','tweets']
tweets_test.head()

,label,tweets
0,POLIT,Global Voices Online Â» Alex Castro: A liberal...
1,POLIT,Do the Conservatives Have a Death Wish? http:/...
2,NOT,@MMFlint I've seen all of your movies and Capi...
3,POLIT,RT @AllianceAlert: * House Dems ask for civili...
4,POLIT,RT @AdamSmithInst Quote of the week: My politi...


In [9]:
x_test = tweets_test['tweets']
y_test = tweets_test['label']
print("Train size : {}, Test Size : {}".format(x_train.shape, x_test.shape))

Train size : (2000,), Test Size : (2004,)


In [23]:
# Converting Str labels to int
y_train = np.where(y_train=="NOT",0,1)
y_test = np.where(y_test=="NOT",0,1)

In [24]:
y_train, y_test

(array([0, 0, 0, ..., 0, 0, 1]), array([1, 1, 0, ..., 1, 1, 0]))